In [3]:
import os
os.chdir("..")
!pwd
!pyenv version
%load_ext autoreload
%autoreload 2

/Users/heather/code/loicmorel/crypto_assistant
crypto_assistant (set by PYENV_VERSION environment variable)


In [4]:
import pandas as pd
import numpy as np
from app.data_collection.datamgt import ohlcv_from_csv_to_df
from tqdm import trange
from scipy import stats
from sklearn.linear_model import LinearRegression

In [5]:
file = 'data/ohlcv_ETHUSDT_1h_from_1502942400000_to_1651507200000.csv'
data = ohlcv_from_csv_to_df(file)
data

,o_ts,Open,High,Low,Close,Volume,c_ts,o_date
0,2017-08-17 04:00:00,301.13,302.57,298.00,301.61,125.66877,2017-08-17 04:59:59.999000064,17395.166667
1,2017-08-17 05:00:00,301.61,303.28,300.00,303.10,377.67246,2017-08-17 05:59:59.999000064,17395.208333
2,2017-08-17 06:00:00,302.40,304.44,301.90,302.68,303.86672,2017-08-17 06:59:59.999000064,17395.250000
3,2017-08-17 07:00:00,302.68,307.96,302.60,307.96,754.74510,2017-08-17 07:59:59.999000064,17395.291667
4,2017-08-17 08:00:00,307.95,309.97,307.00,308.62,150.75029,2017-08-17 08:59:59.999000064,17395.333333
...,...,...,...,...,...,...,...,...
41138,2022-05-02 12:00:00,2795.05,2811.47,2789.28,2795.57,17163.77510,2022-05-02 12:59:59.999000064,19114.500000
41139,2022-05-02 13:00:00,2795.54,2845.10,2778.78,2829.25,64991.93710,2022-05-02 13:59:59.999000064,19114.541667
41140,2022-05-02 14:00:00,2829.25,2852.53,2805.23,2828.00,46120.43640,2022-05-02 14:59:59.999000064,19114.583333
41141,2022-05-02 15:00:00,2827.99,2835.49,2814.10,2827.68,24617.00230,2022-05-02 15:59:59.999000064,19114.625000


In [6]:
from app.indicators import find_swings
data_ = find_swings(data)

In [7]:
from app.indicators import classify_swings
data__ = classify_swings(data_)

In [8]:
data__

,Highs,Lows,Last,Trend,CSLS,Support,Resistance,Strong_lows,Strong_highs,FSL,FSH,LL,HL,HH,LH
0,302.57,298.00,302.57,0.0,0,False,False,0.00,0.00,0.00,0.0,False,False,False,False
1,303.28,300.00,303.28,NaN,0,False,False,0.00,0.00,0.00,0.0,False,False,False,False
2,0.00,298.00,298.00,1.0,0,False,False,0.00,0.00,0.00,0.0,False,False,False,False
3,0.00,298.00,298.00,1.0,1,True,False,298.00,0.00,298.00,0.0,False,True,False,False
4,0.00,298.00,298.00,1.0,2,True,False,298.00,0.00,0.00,0.0,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41138,2874.59,2829.01,2874.59,-1.0,5,False,True,0.00,2874.59,0.00,0.0,False,False,False,False
41139,2874.59,2789.28,2789.28,1.0,0,False,False,0.00,0.00,0.00,0.0,False,False,False,False
41140,2874.59,2789.28,2789.28,1.0,1,True,False,2789.28,0.00,2789.28,0.0,True,False,False,False
41141,2874.59,2789.28,2789.28,1.0,2,True,False,2789.28,0.00,0.00,0.0,False,False,False,False


In [9]:
data__=data__*1

In [10]:
data__

,Highs,Lows,Last,Trend,CSLS,Support,Resistance,Strong_lows,Strong_highs,FSL,FSH,LL,HL,HH,LH
0,302.57,298.00,302.57,0.0,0,0,0,0.00,0.00,0.00,0.0,0,0,0,0
1,303.28,300.00,303.28,NaN,0,0,0,0.00,0.00,0.00,0.0,0,0,0,0
2,0.00,298.00,298.00,1.0,0,0,0,0.00,0.00,0.00,0.0,0,0,0,0
3,0.00,298.00,298.00,1.0,1,1,0,298.00,0.00,298.00,0.0,0,1,0,0
4,0.00,298.00,298.00,1.0,2,1,0,298.00,0.00,0.00,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41138,2874.59,2829.01,2874.59,-1.0,5,0,1,0.00,2874.59,0.00,0.0,0,0,0,0
41139,2874.59,2789.28,2789.28,1.0,0,0,0,0.00,0.00,0.00,0.0,0,0,0,0
41140,2874.59,2789.28,2789.28,1.0,1,1,0,2789.28,0.00,2789.28,0.0,1,0,0,0
41141,2874.59,2789.28,2789.28,1.0,2,1,0,2789.28,0.00,0.00,0.0,0,0,0,0


In [11]:
df=pd.merge(data,data__,left_index=True,right_index=True)

In [12]:
df=df[['o_ts','Open','High','Low','Close','Volume','LL','HL','HH','LH']]

In [13]:
df

,o_ts,Open,High,Low,Close,Volume,LL,HL,HH,LH
0,2017-08-17 04:00:00,301.13,302.57,298.00,301.61,125.66877,0,0,0,0
1,2017-08-17 05:00:00,301.61,303.28,300.00,303.10,377.67246,0,0,0,0
2,2017-08-17 06:00:00,302.40,304.44,301.90,302.68,303.86672,0,0,0,0
3,2017-08-17 07:00:00,302.68,307.96,302.60,307.96,754.74510,0,1,0,0
4,2017-08-17 08:00:00,307.95,309.97,307.00,308.62,150.75029,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
41138,2022-05-02 12:00:00,2795.05,2811.47,2789.28,2795.57,17163.77510,0,0,0,0
41139,2022-05-02 13:00:00,2795.54,2845.10,2778.78,2829.25,64991.93710,0,0,0,0
41140,2022-05-02 14:00:00,2829.25,2852.53,2805.23,2828.00,46120.43640,1,0,0,0
41141,2022-05-02 15:00:00,2827.99,2835.49,2814.10,2827.68,24617.00230,0,0,0,0


In [14]:
df['sum'] = df[['HH','HL','LL','LH']].sum(axis=1)
df['NP']=df['sum'].apply(lambda x:1 if x==0 else 0)
df.drop(columns='sum',inplace=True)

In [15]:
df

,o_ts,Open,High,Low,Close,Volume,LL,HL,HH,LH,NP
0,2017-08-17 04:00:00,301.13,302.57,298.00,301.61,125.66877,0,0,0,0,1
1,2017-08-17 05:00:00,301.61,303.28,300.00,303.10,377.67246,0,0,0,0,1
2,2017-08-17 06:00:00,302.40,304.44,301.90,302.68,303.86672,0,0,0,0,1
3,2017-08-17 07:00:00,302.68,307.96,302.60,307.96,754.74510,0,1,0,0,0
4,2017-08-17 08:00:00,307.95,309.97,307.00,308.62,150.75029,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
41138,2022-05-02 12:00:00,2795.05,2811.47,2789.28,2795.57,17163.77510,0,0,0,0,1
41139,2022-05-02 13:00:00,2795.54,2845.10,2778.78,2829.25,64991.93710,0,0,0,0,1
41140,2022-05-02 14:00:00,2829.25,2852.53,2805.23,2828.00,46120.43640,1,0,0,0,0
41141,2022-05-02 15:00:00,2827.99,2835.49,2814.10,2827.68,24617.00230,0,0,0,0,1


In [16]:
df.set_index('o_ts',inplace=True)

In [39]:
df.to_csv('data/transformed_data/ETHUSDT_1h_transformed.csv')

In [18]:
df[['HH','HL','LL','LH','NP']].value_counts()

HH  HL  LL  LH  NP
0   0   0   0   1     31314
    1   0   0   0      2962
    0   0   1   0      2751
1   0   0   0   0      2082
0   0   1   0   0      2034
dtype: int64

In [19]:
def subsample_sequence(df, length):
    """
    Given the initial dataframe `df`, return a shorter dataframe sequence of length `length`.
    This shorter sequence should be selected at random
    """
    last_possible = df.shape[0] - length
    
    random_start = np.random.randint(0, last_possible)
    df_sample = df[random_start: random_start+length]
    return df_sample

In [20]:
assert subsample_sequence(df, 10).shape == (10, 10)
assert subsample_sequence(df, 10000).shape == (10000, 10)

In [21]:
def split_subsample_sequence(df, length):
    '''Create one single random (X,y) pair'''
    df_subsample = subsample_sequence(df, length)
    y_sample = df_subsample.iloc[length -1][['LL','HL','HH','LH','NP']]
    
    X_sample = df_subsample[0:length -1]
    X_sample = X_sample.values
    return np.array(X_sample), np.array(y_sample)

In [22]:
(X_sample, y_sample) = split_subsample_sequence(df, 10)
assert X_sample.shape == (9,10)
assert y_sample.shape == (5,)

In [23]:
def get_X_y(df, n_sequences, length):
    '''Return a list of samples (X, y)'''
    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(df, length)
        X.append(xi)
        y.append(yi)
        
    X = np.array(X)
    y = np.array(y)
    return X, y

In [27]:
X, y = get_X_y(df, 2000, 21)
print(X.shape)
print(y.shape)

(2000, 20, 10)
(2000, 5)


In [25]:
assert np.isnan(X).sum() == 0

In [26]:
len_ = int(0.8*df.shape[0])
data_train = df[:len_]
data_test = df[len_:]

In [28]:
X_train, y_train = get_X_y(data_train, 2000, 21)
X_test, y_test = get_X_y(data_test, 400, 21)

In [29]:
assert X_train.shape == (2000, 20, 10)

In [30]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.layers.experimental.preprocessing import Normalization

In [31]:


def init_model():
    opt = optimizers.RMSprop(learning_rate=0.01)
    
    normalizer = Normalization()
    normalizer.adapt(X_train)

    model = models.Sequential()
    model.add(normalizer)
    model.add(layers.GRU(units=20, activation='tanh', return_sequences=True))
    model.add(layers.GRU(units=20, activation='tanh', return_sequences=False))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(5, activation='softmax'))

    
    model.compile(loss='categorical_crossentropy', 
                  optimizer=opt, 
                  metrics=['accuracy'])
    
    return model

In [32]:
model = init_model()
model.summary()

2022-05-06 12:50:20.745045: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-06 12:50:20.774383: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, None, 10)          21        
_________________________________________________________________
gru (GRU)                    (None, None, 20)          1920      
_________________________________________________________________
gru_1 (GRU)                  (None, 20)                2520      
_________________________________________________________________
dense (Dense)                (None, 50)                1050      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 255       
Total params: 5,766
Trainable params: 5,745
Non-trainable params: 21
_____________________________________________________

In [33]:
from tensorflow.keras.callbacks import EarlyStopping

es=EarlyStopping(monitor='accuracy', patience=10)

In [34]:
class_weight = {0: 5.,
                1: 5.,
                2: 5.,
                3: 5.,
                4: 1.}

In [35]:
model.fit(X_train, y_train, 
          epochs=75, 
          batch_size=64, 
          verbose=1, 
          callbacks = [es],
          validation_split=0.2,class_weight=class_weight)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/75
25/25 [==============================] - 2s 23ms/step - loss: 2.6953 - accuracy: 0.6938 - val_loss: 0.9676 - val_accuracy: 0.6950
Epoch 2/75
25/25 [==============================] - 0s 8ms/step - loss: 2.4044 - accuracy: 0.6069 - val_loss: 0.9896 - val_accuracy: 0.4925
Epoch 3/75
25/25 [==============================] - 0s 8ms/step - loss: 2.2903 - accuracy: 0.5950 - val_loss: 1.0184 - val_accuracy: 0.4150
Epoch 4/75
25/25 [==============================] - 0s 8ms/step - loss: 2.2396 - accuracy: 0.5394 - val_loss: 0.8462 - val_accuracy: 0.6450
Epoch 5/75
25/25 [==============================] - 0s 8ms/step - loss: 2.1868 - accuracy: 0.5437 - val_loss: 0.9319 - val_accuracy: 0.4625
Epoch 6/75
25/25 [==============================] - 0s 8ms/step - loss: 2.1238 - accuracy: 0.5394 - val_loss: 0.9260 - val_accuracy: 0.5325
Epoch 7/75
25/25 [===

In [36]:
model.evaluate(X_test, y_test)

13/13 [==============================] - 0s 2ms/step - loss: 0.9742 - accuracy: 0.4475


[0.9742377400398254, 0.44749999046325684]

In [37]:
np.round(model.predict_proba(X_test),2)

/Users/heather/.pyenv/versions/3.8.12/envs/crypto_assistant/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


array([[0.  , 0.09, 0.03, 0.42, 0.46],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.29, 0.27, 0.  , 0.17, 0.26],
       ...,
       [0.  , 0.04, 0.16, 0.37, 0.43],
       [0.01, 0.02, 0.  , 0.  , 0.97],
       [0.01, 0.46, 0.15, 0.13, 0.25]], dtype=float32)

In [38]:
model.save('./models/RNN_ETHUSDT_1h.h5')